In [2]:
import os 
os.chdir(r'Q:\sachuriga\Sachuriga_Python/quattrocolo-nwb4fp\src')

In [3]:
from pickle import TRUE
import spikeinterface as si
import spikeinterface.extractors as se
import spikeinterface.postprocessing as post
from nwb4fp.postprocess.Get_positions import load_positions,load_positions_h5,test_positions_h5
from nwb4fp.postprocess.get_potential_merge import get_potential_merge
from spikeinterface.preprocessing import (bandpass_filter,
                                           common_reference,
                                           whiten)
import spikeinterface.exporters as sex

from nwb4fp.preprocess.down_sample_lfp import down_sample_lfp_test

import spikeinterface.qualitymetrics as sqm
from pathlib import Path
import pandas as pd
from nwb4fp.postprocess.extract_wf import wf4unim,divide_wf
import spikeinterface.preprocessing as spre
import numpy as np

from spikeinterface.extractors.neoextractors.openephys import OpenEphysBinaryRecordingExtractor

C:\Users\sachur\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
temp_folder = r"C:/temp_lfp"
path = r"S:\Sachuriga\Ephys_Recording\CR_CA1/65091/65091_2023-08-01_15-16-29_C_phy_k_manual/"
memory_size = 128
global_job_kwargs = dict(n_jobs=32, total_memory=fr"{memory_size}G",mp_context= "spawn",progress_bar=True)
si.set_global_job_kwargs(**global_job_kwargs)

sorting = se.read_phy(folder_path=path, load_all_cluster_properties=True,exclude_cluster_groups = ["noise", "mua"])
temp_path = path.split("_phy")
raw_path = temp_path[0]
#stream_name = 'Record Node 101#OE_FPGA_Acquisition_Board-100.Rhythm Data'
stream_name  = OpenEphysBinaryRecordingExtractor(raw_path,stream_id='0').get_streams(raw_path)[0][0]
print(fr"Before mannual search the stream_name. Auto search result is {stream_name}")
try:
    recording = se.read_openephys(raw_path, stream_name=stream_name, load_sync_timestamps=True)
except AssertionError:
    try:
        stream_name = 'Record Node 102#OE_FPGA_Acquisition_Board-101.Rhythm Data'
        recording = se.read_openephys(raw_path, stream_name=stream_name, load_sync_timestamps=True)
    except AssertionError:
        stream_name = 'Record Node 101#Acquisition_Board-100.Rhythm Data'
        recording = se.read_openephys(raw_path, stream_name=stream_name, load_sync_timestamps=True)

import probeinterface as pi

# from probeinterface import plotting
manufacturer = 'cambridgeneurotech'
probe_name = 'ASSY-236-F'
probe = pi.get_probe(manufacturer, probe_name)
print(probe)
# probe.wiring_to_device('cambridgeneurotech_mini-amp-64')
# map channels to device indices
mapping_to_device = [
    # connector J2 TOP
    41, 39, 38, 37, 35, 34, 33, 32, 29, 30, 28, 26, 25, 24, 22, 20,
    46, 45, 44, 43, 42, 40, 36, 31, 27, 23, 21, 18, 19, 17, 16, 14,
    # connector J1 BOTTOM
    55, 53, 54, 52, 51, 50, 49, 48, 47, 15, 13, 12, 11, 9, 10, 8,
    63, 62, 61, 60, 59, 58, 57, 56, 7, 6, 5, 4, 3, 2, 1, 0
]

probe.set_device_channel_indices(mapping_to_device)
probe.to_dataframe(complete=True).loc[:, ["contact_ids", "shank_ids", "device_channel_indices"]]
probegroup = pi.ProbeGroup()
probegroup.add_probe(probe)

pi.write_prb(f"{probe_name}.prb", probegroup, group_mode="by_shank")
recording_prb = recording.set_probe(probe, group_mode="by_shank")
rec = bandpass_filter(recording_prb, freq_min=600, freq_max=8000)
bad_channel_ids, channel_labels = spre.detect_bad_channels(recording_prb, method='coherence+psd',n_neighbors = 11)
print(fr"removed {bad_channel_ids}")
recording_good_ch= rec.remove_channels(bad_channel_ids)
#recording_good_channels_f = spre.interpolate_bad_channels(rec,bad_channel_ids)

rec_save = common_reference(recording_good_ch, reference='global', operator='median')
rec_w = whiten(rec_save, int_scale=200, mode='local', radius_um=100.0)
sorting.set_property(key='group', values = sorting.get_property("channel_group"))
print(f"get times for raw sorts{sorting.get_times()}")
## step to analyzer
GLOBAL_KWARGS = dict(n_jobs=16, total_memory=fr"{memory_size}G", progress_bar=True, mp_context= "spawn", chunk_size=5000, chunk_duration="1s")
si.set_global_job_kwargs(**GLOBAL_KWARGS)

analyzer = si.create_sorting_analyzer(sorting=sorting, recording=rec_w, format='memory', folder=fr"{temp_folder}",overwrite=True)
we1 = analyzer.compute("random_spikes","waveforms")
we1 = analyzer.compute("noise_levels")
we1 = analyzer.compute("templates")
#get potential merging sorting objects
print("processing potential merge...\n")
## Step to creating analyzers
unit_locations = analyzer.compute(input="unit_locations", method="monopolar_triangulation")
unit_ids = sorting.unit_ids
phy_TRD = Path(path + "_manual_reports")
sex.export_report(sorting_analyzer = analyzer, output_folder=phy_TRD, remove_if_exists=True)


analyzer1 = si.create_sorting_analyzer(sorting=sorting, recording=rec_w, format='memory', folder=fr"{temp_folder}",overwrite=True)
analyzer1.compute("random_spikes","waveforms")
analyzer1.compute("noise_levels")
analyzer1.compute("templates")
#get potential merging sorting objects
print("processing potential merge...\n")
## Step to creating analyzers
unit_locations = analyzer1.compute(input="unit_locations", method="center_of_mass")
unit_ids = sorting.unit_ids
phy_TRD = Path(path + "_manual_mass")
sex.export_report(sorting_analyzer = analyzer1, output_folder=phy_TRD, remove_if_exists=True)


analyzer2 = si.create_sorting_analyzer(sorting=sorting, recording=rec_w, format='memory', folder=fr"{temp_folder}",overwrite=True)
analyzer2.compute("random_spikes","waveforms")
analyzer2.compute("noise_levels")
analyzer2.compute("templates")
#get potential merging sorting objects
print("processing potential merge...\n")
## Step to creating analyzers
unit_locations = analyzer2.compute(input="unit_locations", method="grid_convolution")
unit_ids = sorting.unit_ids
phy_TRD = Path(path + "_manual_reports_grid")
sex.export_report(sorting_analyzer = analyzer2, output_folder=phy_TRD, remove_if_exists=True)

Before mannual search the stream_name. Auto search result is Record Node 102#OE_FPGA_Acquisition_Board-101.Rhythm Data
ASSY-236-F - cambridgeneurotech - 64ch - 6shanks
removed ['CH12' 'CH18']
get times for raw sortsNone


estimate_sparsity (workers: 12 processes): 100%|██████████| 7302/7302 [00:05<00:00, 1416.48it/s]
c:\Users\sachur\AppData\Local\anaconda3\envs\test1\Lib\site-packages\spikeinterface\core\recording_tools.py:786: UserWarning: get_noise_levels(recording, num_chunks_per_segment=20) is deprecated
Now, you need to use get_noise_levels(recording, random_slices_kwargs=dict(num_chunks_per_segment=20, chunk_size=1000))
Please read get_random_recording_slices() documentation for more options.
  warnings.warn(msg)
noise_level (workers: 12 processes): 100%|██████████| 20/20 [00:01<00:00, 10.36it/s]
estimate_templates_with_accumulator (workers: 12 processes): 100%|██████████| 7302/7302 [00:10<00:00, 689.41it/s] 


processing potential merge...

export_report(): spike_amplitudes will not be exported. Use sorting_analyzer.compute('spike_amplitudes') if you want to include them.
export_report(): quality metrics will not be exported. Use sorting_analyzer.compute('quality_metrics') if you want to include them.
export_report(): correlograms will not be exported. Use sorting_anlyzer.compute('correlograms') if you want to include them.


c:\Users\sachur\AppData\Local\anaconda3\envs\test1\Lib\site-packages\spikeinterface\widgets\unit_waveforms.py:182: UserWarning: templates_percentile_shading can only be used if the 'waveforms' extension is available. Settimg templates_percentile_shading to None.
  warn(
estimate_sparsity (workers: 12 processes): 100%|██████████| 7302/7302 [00:04<00:00, 1691.51it/s] 
estimate_templates_with_accumulator (workers: 12 processes): 100%|██████████| 7302/7302 [00:08<00:00, 833.87it/s] 


processing potential merge...

export_report(): spike_amplitudes will not be exported. Use sorting_analyzer.compute('spike_amplitudes') if you want to include them.
export_report(): quality metrics will not be exported. Use sorting_analyzer.compute('quality_metrics') if you want to include them.
export_report(): correlograms will not be exported. Use sorting_anlyzer.compute('correlograms') if you want to include them.


c:\Users\sachur\AppData\Local\anaconda3\envs\test1\Lib\site-packages\spikeinterface\widgets\unit_waveforms.py:182: UserWarning: templates_percentile_shading can only be used if the 'waveforms' extension is available. Settimg templates_percentile_shading to None.
  warn(
estimate_sparsity (workers: 12 processes): 100%|██████████| 7302/7302 [00:04<00:00, 1630.62it/s]
estimate_templates_with_accumulator (workers: 12 processes): 100%|██████████| 7302/7302 [00:07<00:00, 1007.30it/s]


processing potential merge...

export_report(): spike_amplitudes will not be exported. Use sorting_analyzer.compute('spike_amplitudes') if you want to include them.
export_report(): quality metrics will not be exported. Use sorting_analyzer.compute('quality_metrics') if you want to include them.
export_report(): correlograms will not be exported. Use sorting_anlyzer.compute('correlograms') if you want to include them.


c:\Users\sachur\AppData\Local\anaconda3\envs\test1\Lib\site-packages\spikeinterface\widgets\unit_waveforms.py:182: UserWarning: templates_percentile_shading can only be used if the 'waveforms' extension is available. Settimg templates_percentile_shading to None.
  warn(


In [4]:
from nwb4fp.preprocess.down_sample_lfp import down_sample_lfp_test
temp_folder = r"C:/temp_lfp"
path = r"S:\Sachuriga\Ephys_Recording\CR_CA1/65091/65091_2023-08-01_15-16-29_C_phy_k_manual"
raw_path = r"S:\Sachuriga\Ephys_Recording\CR_CA1/65091/65091_2023-08-01_15-16-29_C"
down_sample_lfp_test(path,raw_path)

Merging step_Before mannual search the stream_name. Auto search result is Record Node 102#OE_FPGA_Acquisition_Board-101.Rhythm Data
LFP downsampling steps. Auto search result is Record Node 102#OE_FPGA_Acquisition_Board-101.Rhythm Data
ASSY-236-F - cambridgeneurotech - 64ch - 6shanks
bad channels are [11, 17]
bad channels labels ['good' 'good' 'good' 'good' 'good' 'good' 'good' 'good' 'good' 'good'
 'good' 'dead' 'good' 'good' 'good' 'good' 'good' 'dead' 'good' 'good'
 'good' 'good' 'good' 'good' 'good' 'good' 'good' 'good' 'good' 'good'
 'good' 'good' 'good' 'good' 'good' 'good' 'good' 'good' 'good' 'good'
 'good' 'good' 'good' 'good' 'good' 'good' 'good' 'good' 'good' 'good'
 'good' 'good' 'good' 'good' 'good' 'good' 'good' 'good' 'good' 'good'
 'good' 'good' 'good' 'good']
['CH1' 'CH2' 'CH3' 'CH4' 'CH5' 'CH6' 'CH7' 'CH8' 'CH9' 'CH10' 'CH11'
 'CH12' 'CH13' 'CH14' 'CH15' 'CH16' 'CH17' 'CH18' 'CH19' 'CH20' 'CH21'
 'CH22' 'CH23' 'CH24' 'CH25' 'CH26' 'CH27' 'CH28' 'CH29' 'CH30' 'CH31'
 '

write_binary_recording (workers: 12 processes): 100%|██████████| 305/305 [02:40<00:00,  1.90it/s]


processing lfp data...
write_binary_recording 
engine=process - n_jobs=12 - samples_per_chunk=5,000 - chunk_memory=625.00 KiB - total_memory=7.32 MiB - chunk_duration=4.00s


write_binary_recording (workers: 12 processes): 100%|██████████| 305/305 [00:36<00:00,  8.26it/s]


In [5]:

phy_TRD = Path(path + "_manual_reports")
sex.export_report(sorting_analyzer = analyzer, output_folder=phy_TRD, remove_if_exists=True)

export_report(): spike_amplitudes will not be exported. Use sorting_analyzer.compute('spike_amplitudes') if you want to include them.
export_report(): quality metrics will not be exported. Use sorting_analyzer.compute('quality_metrics') if you want to include them.
export_report(): correlograms will not be exported. Use sorting_anlyzer.compute('correlograms') if you want to include them.


c:\Users\sachur\AppData\Local\anaconda3\envs\test1\Lib\site-packages\spikeinterface\widgets\unit_waveforms.py:182: UserWarning: templates_percentile_shading can only be used if the 'waveforms' extension is available. Settimg templates_percentile_shading to None.
  warn(


In [5]:
analyzer1 = si.create_sorting_analyzer(sorting=sorting, recording=rec_w, format='memory', folder=fr"{temp_folder}",overwrite=True)
analyzer1.compute("random_spikes","waveforms")
analyzer1.compute("noise_levels")
analyzer1.compute("templates")
#get potential merging sorting objects
print("processing potential merge...\n")
## Step to creating analyzers
unit_locations = analyzer1.compute(input="unit_locations", method="center_of_mass")
unit_ids = sorting.unit_ids
phy_TRD = Path(path + "_manual_mass")
#sex.export_report(sorting_analyzer = analyzer1, output_folder=phy_TRD, remove_if_exists=True)


estimate_sparsity (workers: 12 processes): 100%|██████████| 7302/7302 [00:04<00:00, 1669.72it/s]
estimate_templates_with_accumulator (workers: 12 processes): 100%|██████████| 7302/7302 [00:07<00:00, 981.76it/s] 


processing potential merge...



In [25]:
unit_loc_mt=[]
unit_loc_mt.append(list(unit_ids))
unit_loc_mt.append(list(unit_locations.data['unit_locations'].T))
np.save('unit_loc_mt.npy', unit_loc_mt) # type: ignore

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [26]:
unit_loc_mt

[[6, 7, 10, 11, 25, 29, 2, 3, 9, 14, 18, 20, 33, 30],
 [array([207.16263843, 202.82946004, 209.52251038, 203.74349041,
         613.09690519, 604.14581035,  11.69277035,   5.23775735,
         211.61895249, 407.45807776, 406.45500903, 403.39049644,
         809.82154077, 605.02105199]),
  array([ 47.12129809,  54.47187184,  55.11142142,  41.66844636,
          34.90147152,  14.78394834,  32.78757262,  23.93706088,
          54.14107789,  62.0514003 ,  33.5406549 ,  35.22035646,
          52.89583812, 105.09188348])]]

In [8]:
analyzer2 = si.create_sorting_analyzer(sorting=sorting, recording=rec_w, format='memory', folder=fr"{temp_folder}",overwrite=True)
analyzer2.compute("random_spikes","waveforms")
analyzer2.compute("noise_levels")
analyzer2.compute("templates")
#get potential merging sorting objects
print("processing potential merge...\n")
## Step to creating analyzers
unit_locations = analyzer2.compute(input="unit_locations", method="grid_convolution")
unit_ids = sorting.unit_ids
phy_TRD = Path(path + "_manual_reports_grid")
sex.export_report(sorting_analyzer = analyzer2, output_folder=phy_TRD, remove_if_exists=True)

estimate_sparsity (workers: 12 processes): 100%|██████████| 7705/7705 [00:04<00:00, 1641.66it/s] 
estimate_templates_with_accumulator (workers: 12 processes): 100%|██████████| 7705/7705 [00:10<00:00, 765.98it/s] 


processing potential merge...

export_report(): spike_amplitudes will not be exported. Use sorting_analyzer.compute('spike_amplitudes') if you want to include them.
export_report(): quality metrics will not be exported. Use sorting_analyzer.compute('quality_metrics') if you want to include them.
export_report(): correlograms will not be exported. Use sorting_anlyzer.compute('correlograms') if you want to include them.


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'S:\\Sachuriga\\Ephys_Recording\\CR_CA1\\65588\\65588_2024-03-04_15-44-37_A_phy_k_manual\\_manual_reports_grid'

In [9]:
sex.export_report(sorting_analyzer = analyzer2, output_folder=phy_TRD, remove_if_exists=True)

export_report(): spike_amplitudes will not be exported. Use sorting_analyzer.compute('spike_amplitudes') if you want to include them.
export_report(): quality metrics will not be exported. Use sorting_analyzer.compute('quality_metrics') if you want to include them.
export_report(): correlograms will not be exported. Use sorting_anlyzer.compute('correlograms') if you want to include them.


c:\Users\sachur\AppData\Local\anaconda3\envs\test1\Lib\site-packages\spikeinterface\widgets\unit_waveforms.py:182: UserWarning: templates_percentile_shading can only be used if the 'waveforms' extension is available. Settimg templates_percentile_shading to None.
  warn(
